In [31]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta
import sqlite3


In [32]:
fake = Faker()
np.random.seed(42)

In [33]:
#Parameters
num_players = 1000
num_bets = 10000
num_games = 50
num_transactions = 5000
countries = ['USA', 'Canada', 'UK', 'Germany', 'France', 'Italy', 'Spain', 'Australia', 'India', 'China']
devices = ['Mobile', 'Desktop', 'Tablet']
game_types = ['Slots', 'Table Games', 'Live Casino', 'Sports Betting']
providers = ['NetEnt', 'Evolution', 'Microgaming', 'Playtech']
volatility_levels = ['low', 'medium', 'high']
transaction_types = ['deposit', 'withdrawal', 'bonus', 'refund']
payment_methods = ['credit_card', 'paypal', 'bank_transfer', 'crypto']

In [34]:
#generate player data
players = []
for i in range(1, num_players + 1):
    players.append({
        'player_id': i,
        'registration_date': fake.date_between(start_date='-2y', end_date='today'),
        'country': random.choice(countries),
        'age': random.randint(18, 70),
        'device': random.choice(devices)
    })

players_df = pd.DataFrame(players)

In [35]:
# -------- Introduce Broken Data in Players --------
# Randomly make 5% of the 'age' column invalid (e.g., set negative or unrealistic values)
players_df.loc[players_df.sample(frac=0.05).index, 'age'] = -1

# Randomly set 5% of 'country' to None (null) to simulate missing data
players_df.loc[players_df.sample(frac=0.05).index, 'country'] = None

In [36]:
games = []
for game_id in range(1, num_games + 1):
    games.append({
        'game_id': game_id,
        'game_type': random.choice(game_types),
        'provider': random.choice(providers),
        'volatility': random.choice(volatility_levels),
        'release_date': fake.date_between(start_date='-5y', end_date='today'),
        'rtp': random.uniform(0.85, 0.98)
    })

games_df = pd.DataFrame(games)

In [37]:
# Randomly set 10% of 'rtp' values to a string or invalid number
games_df.loc[games_df.sample(frac=0.20).index, 'rtp'] = 'invalid'

C:\Users\Fernanda\AppData\Local\Temp\ipykernel_16436\1110509575.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'invalid' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  games_df.loc[games_df.sample(frac=0.20).index, 'rtp'] = 'invalid'


In [38]:
#Generate bet data
bets = []
for i in range(1, num_bets + 1):
    player = random.choice(players_df['player_id'])
    game_id = random.choice(games_df['game_id'])
    bet_time = fake.date_time_between(start_date='-1y', end_date='now')
    bet_amount = round(random.uniform(1, 1000), 2)
    win_amount = round(bet_amount * random.choice([0, 0, 0, 1, 2, 1.5, 5]), 2)
    bets.append({
        'bet_id': i,
        'player_id': player,
        'bet_time': bet_time,
        'game_id': game_id,
        'bet_amount': bet_amount,
        'win_amount': win_amount
    })

bets_df = pd.DataFrame(bets)

In [39]:
# -------- Introduce Broken Data in Bets --------
# Randomly add negative bets (which should be impossible in the real world)
bets_df.loc[bets_df.sample(frac=0.02).index, 'bet_amount'] = -abs(bets_df['bet_amount'])

In [40]:
#transactions
transactions =[]
for trans_id in range(1, num_transactions +1):
    player_id = random.choice(players_df['player_id'])
    transaction_time = fake.date_time_between(start_date='-1y', end_date='now')
    transaction_type = random.choices(transaction_types, weights=[0.5, 0.2, 0.2, 0.1])
    transaction_amount = round(random.uniform(1, 1000), 2)
    payment_method = random.choice(payment_methods)

    if transaction_type in ['withdrawal', 'refund']:
        transaction_amount = -transaction_amount
    
    transactions.append({
        'transactio_id': trans_id,
        'player_id': player_id,
        'transaction_time': transaction_time,
        'transaction_type': transaction_type,
        'transaction_amount': transaction_amount,
        'payment_method': payment_method
    })
    transactions_df = pd.DataFrame(transactions)

In [41]:
# Randomly make 50% of the 'amount' column NaN (missing data)
transactions_df.loc[transactions_df.sample(frac=0.50).index, 'amount'] = np.nan

In [42]:
import json
transactions_df['transaction_type'] = transactions_df['transaction_type'].apply(lambda x: json.dumps(x[0]))

In [43]:
#Generate game sessions
sessions = []
for session_id in range(1, num_bets + 1):
    player = random.choice(players_df['player_id'])
    game = random.choice(games_df['game_id'])
    session_start = fake.date_time_between(start_date='-1y', end_date='now')
    session_end = session_start + timedelta(minutes=random.randint(5, 120))
    bet_amount = round(random.uniform(1, 1000), 2)
    win_amount = round(bet_amount * random.choice([0, 0, 0, 1, 2, 1.5, 5]), 2)
    
    sessions.append({
        'session_id': session_id,
        'player_id': player,
        'game_id': game,
        'session_start': session_start,
        'session_end': session_end,
        'bet_amount': bet_amount,
        'win_amount': win_amount
    })

sessions_df = pd.DataFrame(sessions)

In [44]:
# Randomly duplicate some session rows
sessions_df = pd.concat([sessions_df, sessions_df.sample(frac=0.05)])

In [45]:
campaign_names = ['Welcome Bonus', 'Free Spins', 'VIP Reward', 'Cashback Offer']
num_campaigns = 800
campaigns = []
for campaign_id in range(1, num_campaigns + 1):
    player_id = random.choice(players_df['player_id'])
    assigned_at = fake.date_time_between(start_date='-1y', end_date='now')
    game_id = random.choice(games_df['game_id'])
    campaigns.append({
        'campaign_id': campaign_id,
        'player_id': player_id,
        'game_id': game_id,!
        'name': random.choice(campaign_names),
        'reward': round(random.uniform(5, 100), 2),
        'assigned_at': assigned_at,
        'redeemed': random.choice([True, False])
    })
campaigns_df = pd.DataFrame(campaigns)

SyntaxError: invalid syntax (1043537214.py, line 11)

In [ ]:
# Randomly insert missing player_id (which doesn't exist)
campaigns_df.loc[campaigns_df.sample(frac=0.03).index, 'player_id'] = -1

In [ ]:
# -------- Save to SQLite database --------
conn = sqlite3.connect("gaming_data_full.db")  # Saves in current working directory
players_df.to_sql("players", conn, index=False, if_exists="replace")
bets_df.to_sql("bets", conn, index=False, if_exists="replace")
games_df.to_sql("games", conn, index=False, if_exists="replace")
transactions_df.to_sql("transactions", conn, index=False, if_exists="replace")
sessions_df.to_sql("sessions", conn, index=False, if_exists="replace")
campaigns_df.to_sql("campaigns", conn, index=False, if_exists="replace")

800

In [ ]:
# Connect to the SQLite database
conn_sqlite = sqlite3.connect("gaming_data_full.db")

# List of tables to export
tables = ["players", "bets", "games", "transactions", "sessions", "campaigns"]

# Export each table to a CSV
for table in tables:
    df = pd.read_sql(f"SELECT * FROM {table}", conn_sqlite)
    df.to_csv(f"{table}.csv", index=False)

# Close SQLite connection
conn_sqlite.close()

In [ ]:
campaigns_df


,campaign_id,player_id,name,reward,assigned_at,redeemed
0,1,843,VIP Reward,76.32,2025-03-20 03:09:14,False
1,2,117,Free Spins,98.89,2025-02-08 03:24:13,False
2,3,292,Welcome Bonus,17.87,2024-05-22 14:19:31,True
3,4,785,Free Spins,52.72,2024-06-22 17:02:23,True
4,5,-1,Free Spins,5.59,2025-03-17 14:00:20,False
...,...,...,...,...,...,...
795,796,130,Welcome Bonus,73.24,2024-06-09 12:46:34,False
796,797,382,Welcome Bonus,66.90,2024-11-12 23:17:38,False
797,798,229,VIP Reward,17.02,2024-09-21 18:05:52,False
798,799,538,Welcome Bonus,67.54,2024-08-14 23:51:04,True
